# Model for ETFA 2025

## Imports

In [ ]:
import os

import pvlib
from pvlib.location import Location
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
from pvlib.pvsystem import PVSystem
from pvlib.modelchain import ModelChain

import torch
from torch import nn
import torch.nn.functional as F

import pyro
import pyro.distributions as dist
from pyro.nn import PyroSample
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.nn import PyroModule
from pyro.infer import SVI, Trace_ELBO
from pyro.infer import Predictive

import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

## Debug activation

In [ ]:
DEBUG = True

## System definition

In [ ]:
# Define the location of the PV system
latitude = 41.499779
longitude = 2.111753
timezone = 'Europe/Madrid'
altitude = 133
location_name = 'UAB'
location = Location(latitude=latitude, longitude=longitude,
                    tz=timezone, altitude=altitude, name=location_name)

# Define temperature model parameters
temperature_parameters = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']


# Define the PV system (specialy module)

sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
CEC_inverters = pvlib.pvsystem.retrieve_sam('CECInverter')

module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
inverter = CEC_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']

if DEBUG and True:
    print(f'Module for testing: \n{module}')


module_efficiency = module['Vmpo'] * module['Impo'] / (1000*module['Area'])
module_area = module['Area']

if DEBUG and True:
    print(f'Efficiency of the module: {module_efficiency*100} %')

azimuth_south_centered = 0
system = PVSystem(surface_tilt=30, surface_azimuth=(180-azimuth_south_centered),
                  module_parameters=module, inverter_parameters=inverter,
                  temperature_model_parameters=temperature_parameters)
if DEBUG and True:
    print(f'System for testing: \n{system}')


# Generate the model chain
modelchain = ModelChain(system, location)
if DEBUG and True:
    print(f'Model chain for testing: \n{modelchain}')


# Define the time period for the simulation
start_time = '2024-01-01'
end_time = '2024-11-27 23:00:00'
times = pd.date_range(start=start_time, end=end_time,
                      freq='1h', tz=location.tz)

# Generate the weather data
weather = location.get_clearsky(times)
if DEBUG and True:
    print(f'Weather data for testing: \n{weather}')
    weather.plot()
    plt.show()

## Ground truth (pvlib)

In [ ]:
modelchain.run_model(weather=weather)
gth_power = modelchain.results.dc['p_mp']
gth_power.name = "gth_power"
if DEBUG and True:
    print(f'GTH: \n{gth_power}')
    gth_power.plot(figsize=(32, 9))
    plt.show()

## Model 0

P = Area · eff · solar_power

No training involved

In [ ]:
m0_power_W = module['Area'] * module_efficiency * weather['ghi']
m0_power_W.name = "model_0"
if DEBUG and True:
    print(f'Model 0: \n{m0_power_W}')
    m0_power_W.plot(figsize=(32, 9))
    plt.show()

## Model 5: Bayessian aproximation

In [ ]:
torch.set_num_threads(os.cpu_count())
print(f"Using {os.cpu_count()} CPU threads")

### Model

In [ ]:
# 0) Define lists for storing results
# for hourly predictions
m5_hourly_power_pred_W_list = []
hourly_ci_lower_list = []
hourly_ci_upper_list = []
hourly_weight_means = []
hourly_weight_upper_list = []
hourly_weight_lower_list = []


# for daily predictions
m5_daily_power_pred_W_list = []
daily_ci_lower_list = []
daily_ci_upper_list = []
daily_weight_means = []
daily_weight_upper_list = []
daily_weight_lower_list = []

# 1) Define data
x_data = torch.tensor((weather['ghi'].values*module_area), dtype=torch.float32)
y_data = torch.tensor(gth_power.values, dtype=torch.float32)

if DEBUG and False:
    print(f'X shape: {x_data.shape}') # Output: torch.Size([337])
    print(f'Y shape: {y_data.shape}') # Output: torch.Size([337])
    print(f'X sample shape: {x_data[0].shape}') # Output: torch.Size([])
    print(f'Y sample shape: {y_data[0].shape}') # Output: torch.Size([])
    #print(f'X data: {x_data}')
    #print(f'Y data: {y_data}')

if DEBUG and False:
    print(f'Module efficiency: {module_efficiency}')

# 2) Define model
class BayesianRegression(PyroModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_features, out_features, bias=False)
        self.linear.weight = PyroSample(dist.Normal(module_efficiency, 0.01).expand([out_features, in_features]).to_event(2))

    def forward(self, x, y=None):
        sigma = pyro.sample("sigma", dist.Uniform(0., 0.01))
        raw_mean = self.linear(x).squeeze(-1)

        # Enforce non-negative predictions
        mean = F.relu(raw_mean)

        with pyro.plate("data", x.shape[0]):
            pyro.sample("obs", dist.Normal(mean, sigma), obs=y)
        return mean
   
model = BayesianRegression(1, 1)
guide = AutoDiagonalNormal(model, 
                           init_loc_fn=pyro.infer.autoguide.initialization.init_to_value(values={"linear.weight": torch.tensor([[module_efficiency]])}),
                           init_scale=0.1)
adam = pyro.optim.Adam({"lr": 0.001})
svi = SVI(model, guide, adam, loss=Trace_ELBO())
predictive = Predictive(model, guide=guide, num_samples=800,
                            return_sites=("_RETURN",), parallel=False)


# 3) Training loop
# 3.0) Initial step to initialize the model
svi.step(x_data[0].unsqueeze(0), y_data[0].unsqueeze(0))
for i in range(x_data.shape[0]):
    # 3.1) get current sample
    x = x_data[i].unsqueeze(0)
    y = y_data[i].unsqueeze(0)

    # 3.2) get the current prediction
    # hourly prediction
    hour_samples = predictive(x.unsqueeze(0)) 
    hour_sampled_preds = hour_samples["_RETURN"].squeeze(1) 
    hour_mean_prediction = hour_sampled_preds.mean().item()
    hour_lower = torch.quantile(hour_sampled_preds, 0.05).item()
    hour_upper = torch.quantile(hour_sampled_preds, 0.95).item()
    # daily prediction
    if i % 24 == 0 and ((i+23) < (x_data.shape[0])):
        day_mean_prediction = []
        day_lower = []
        day_upper = []
        for j in range(24):
            day_samples = predictive(x_data[i+j].unsqueeze(0))
            day_sampled_preds = day_samples["_RETURN"]
            day_mean_prediction.append(day_sampled_preds.mean().item())
            day_lower.append(torch.quantile(day_sampled_preds, 0.05).item())
            day_upper.append(torch.quantile(day_sampled_preds, 0.95).item())

    # 3.3) train the model
    loss = svi.step(x, y)

    # 3.4) get current parameters
    current_efficiency = guide.quantiles([0.5])["linear.weight"].detach()  
    current_upper_efficiency = guide.quantiles([0.95])["linear.weight"].detach()
    current_lower_efficiency = guide.quantiles([0.05])["linear.weight"].detach() 

    # 3.5) store the results
    # hourly save
    m5_hourly_power_pred_W_list.append(hour_mean_prediction)
    hourly_ci_lower_list.append(hour_lower)
    hourly_ci_upper_list.append(hour_upper)
    hourly_weight_means.append(current_efficiency[0].item())
    hourly_weight_upper_list.append(current_upper_efficiency[0].item())
    hourly_weight_lower_list.append(current_lower_efficiency[0].item())
    # daily save
    if i % 24 == 0 and ((i+23) < (x_data.shape[0])):
        m5_daily_power_pred_W_list.extend(day_mean_prediction)
        daily_ci_lower_list.extend(day_lower)
        daily_ci_upper_list.extend(day_upper)
        daily_weight_means.append(current_efficiency[0].item())
        daily_weight_upper_list.append(current_upper_efficiency[0].item())
        daily_weight_lower_list.append(current_lower_efficiency[0].item())

    # Control the progress
    if (i % 10 == 0) and DEBUG and True:
        temp_eff = y.item()/x.item() if x.item() != 0 else 'X'
        print(f'Iteration {i}/{x_data.shape[0]} with loss: {loss:.4f}')
        print(f'Efficiency prediction: {current_lower_efficiency.item():.4f} - {current_efficiency.item():.4f} - {current_upper_efficiency.item():.4f} GTH: {temp_eff}')
        print(f'Interval: {hour_lower:.4f} - {hour_mean_prediction:.4f} - {hour_upper:.4f} GTH: {y.item():.4f}')

### Ground truth values

In [ ]:
# Real efficiency evolution
real_efficiency = y_data / x_data
real_efficiency = pd.Series(real_efficiency.numpy(), index=weather.index, name="real_efficiency")
real_efficiency = real_efficiency.resample('D').mean()

### Plots config

In [ ]:
import scienceplots

plt.style.use(["science", "no-latex"])                                                                                                #
plt.rcParams.update({
    "font.size": 28,           # Base font size for all text in the plot.
    "axes.titlesize": 12,      # Title size for axes.
    "axes.labelsize": 12,      # Axis labels size.
    "xtick.labelsize": 22,     # X-axis tick labels size.
    "ytick.labelsize": 22,     # Y-axis tick labels size.
    "legend.fontsize": 100,     # Legend font size.
    "figure.titlesize": 32,    # Overall figure title size.
})                                                                                                                             #
sns.set_theme(style="whitegrid", context="paper") # Set seaborn theme for additional aesthetics and context.                          #


### Plots for paper

In [ ]:
# Prepare data
## Model 5 predictions
power_output_pred_hourly_W = pd.Series(m5_daily_power_pred_W_list, index=gth_power.index, name="power_output_predicted")
power_upper_hourly_W = pd.Series(daily_ci_upper_list, index=gth_power.index, name="power_output_upper")
power_lower_hourly_W = pd.Series(daily_ci_lower_list, index=gth_power.index, name="power_output_lower")

power_output_pred_daily_W = power_output_pred_hourly_W.resample('D').sum()
power_upper_daily_W = power_upper_hourly_W.resample('D').sum()
power_lower_daily_W = power_lower_hourly_W.resample('D').sum()
ground_truth_daily_W = gth_power.resample('D').sum()

## Model 5 efficiency estimations
efficiency_pred_daily = pd.Series(daily_weight_means, index=power_lower_daily_W.index, name="efficiency_predicted")
efficiency_upper_daily = pd.Series(daily_weight_upper_list, index=power_lower_daily_W.index, name="efficiency_upper")
efficiency_lower_daily = pd.Series(daily_weight_lower_list, index=power_lower_daily_W.index, name="efficiency_lower")

## Remove the last month from the data
"""
last_month = power_output_pred_daily_W.index[-1].to_period('M')
filtered_index = power_output_pred_daily_W.index[power_output_pred_daily_W.index.to_period('M') != last_month]

power_output_pred_daily_W = power_output_pred_daily_W.loc[filtered_index]
power_upper_daily_W = power_upper_daily_W.loc[filtered_index]
power_lower_daily_W = power_lower_daily_W.loc[filtered_index]
ground_truth_daily_W = ground_truth_daily_W.loc[filtered_index]

efficiency_pred_daily = efficiency_pred_daily.loc[filtered_index]
efficiency_upper_daily = efficiency_upper_daily.loc[filtered_index]
efficiency_lower_daily = efficiency_lower_daily.loc[filtered_index]
"""

## Model 0 predictions
power_output_pred_hourly_W_m0 = pd.Series(m0_power_W, index=gth_power.index, name="power_output_predicted_m0")
power_output_pred_daily_W_m0 = power_output_pred_hourly_W_m0.resample('D').sum()

## Model 0 efficiency estimations
efficiency_pred_daily_m0 = pd.Series([module_efficiency] * len(power_output_pred_daily_W_m0),
                                      index=power_output_pred_daily_W_m0.index,
                                      name="efficiency_predicted_m0")



def plot3days(): # Done
    # 1) Concatenate the 3 days in a pd Series
    middle_day_number = len(power_output_pred_hourly_W) // (2*24)
    last_day_number = len(power_output_pred_hourly_W) // 24

    first_day = power_output_pred_hourly_W[0:24]
    middle_day = power_output_pred_hourly_W[24*middle_day_number:24*(middle_day_number+1)]
    last_day = power_output_pred_hourly_W[-24:]
    three_days = pd.concat([first_day, middle_day, last_day])
    three_days_upper = pd.concat([power_upper_hourly_W[0:24], power_upper_hourly_W[24*middle_day_number:24*(middle_day_number+1)], power_upper_hourly_W[-24:]])
    three_days_lower = pd.concat([power_lower_hourly_W[0:24], power_lower_hourly_W[24*middle_day_number:24*(middle_day_number+1)], power_lower_hourly_W[-24:]])
    # 2) Same for the ground truth
    gth_power_first_day = gth_power[0:24]
    gth_power_middle_day = gth_power[24*middle_day_number:24*(middle_day_number+1)]
    gth_power_last_day = gth_power[-24:]
    gth_power_three_days = pd.concat([gth_power_first_day, gth_power_middle_day, gth_power_last_day])
    # Same for the Model 0
    m0_power_first_day = power_output_pred_hourly_W_m0[0:24]
    m0_power_middle_day = power_output_pred_hourly_W_m0[24*middle_day_number:24*(middle_day_number+1)]
    m0_power_last_day = power_output_pred_hourly_W_m0[-24:]
    m0_power_three_days = pd.concat([m0_power_first_day, m0_power_middle_day, m0_power_last_day])

    # 3) Create a new index: Day 1-00 ... Day 3-23
    new_index = [f"Day {d+1}-{h:02d}" for d in range(3) for h in range(24)]
    min_len = min(len(new_index), len(three_days))
    new_index = new_index[:min_len]
    three_days = three_days.iloc[:min_len]
    three_days_upper = three_days_upper.iloc[:min_len]
    three_days_lower = three_days_lower.iloc[:min_len]
    gth_power_three_days = gth_power_three_days.iloc[:min_len]
    three_days.index = new_index
    three_days_upper.index = new_index
    three_days_lower.index = new_index
    gth_power_three_days.index = new_index

    # 4) Plot with less x-ticks and thick lines between days
    plt.figure(figsize=(12, 5))
    plt.plot(three_days.index, three_days.values, label='Predicted Power', color='tab:blue', linewidth=2)
    plt.fill_between(three_days.index, three_days_lower.values, three_days_upper.values, color='tab:blue', alpha=0.3, label='90\\% Confidence Interval')
    plt.plot(gth_power_three_days.index, gth_power_three_days.values, label='Ground Truth', color='tab:green', linewidth=2, linestyle='--')
    plt.plot(three_days.index, m0_power_three_days.values, label='Reference', color='tab:red', linewidth=2, linestyle=':')

    # Set fewer x-ticks: only show 0, 12, 23 for each day
    xticks = [0, 12, 24, 36, 48, 60, 71]
    xtick_labels = [
        "D1-00H", "D1-12H", 
        f"D{middle_day_number}-00H", f"D{middle_day_number}-12H",
        f"D{last_day_number}-00H", f"D{last_day_number}-12H", f"D{last_day_number}-23"
    ]
    plt.xticks(xticks, xtick_labels, rotation=0)

    # Add thick vertical lines to separate days
    for sep in [24, 48]:
        plt.axvline(sep - 0.5, color='black', linewidth=3, alpha=0.5)

    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    plt.title("Hourly Power Output: Model Prediction vs Ground Truth (3 Sample Days)", fontsize=24)
    plt.xlabel("Day-Hour", fontsize=18)
    plt.xticks(fontsize=14)
    plt.ylabel("Power (W)", fontsize=18)
    plt.yticks(fontsize=14)
    plt.grid(True, axis='y')
    plt.legend(fontsize=12, frameon=True, facecolor='white', edgecolor='black')
    plt.tight_layout()
    plt.show()

def plotOutputEvolution(): # Done
    plt.figure(figsize=(12, 5))
    plt.plot(power_output_pred_daily_W.index, power_output_pred_daily_W, label="Predicted Power", color='tab:blue', linewidth=2)
    plt.fill_between(power_upper_daily_W.index, power_lower_daily_W, power_upper_daily_W, color='tab:blue', alpha=0.3, label='90\\% Confidence Interval')
    plt.plot(ground_truth_daily_W.index, ground_truth_daily_W, label="Ground Truth", color='tab:green', linewidth=2, linestyle='--')
    plt.plot(power_output_pred_daily_W_m0.index, power_output_pred_daily_W_m0, label="Reference", color='tab:red', linewidth=2, linestyle=':')
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    plt.title("Daily Power Output: Model Prediction vs Ground Truth", fontsize=24)
    plt.xlabel("Day-Hour", fontsize=18)
    plt.xticks(fontsize=14)
    plt.ylabel("Power (W)", fontsize=18)
    plt.yticks(fontsize=14)
    plt.grid(True, axis='y')
    plt.legend(fontsize=12, frameon=True, facecolor='white', edgecolor='black')
    plt.show()

def plotEfficiencyEvolution():
    plt.figure(figsize=(12, 5))
    plt.plot(efficiency_pred_daily.index, efficiency_pred_daily, label="Predicted Efficiency", color='tab:blue', linewidth=2)
    plt.fill_between(efficiency_upper_daily.index, efficiency_lower_daily, efficiency_upper_daily, color='tab:blue', alpha=0.3, label='90\\% Confidence Interval')
    plt.plot(real_efficiency.index, real_efficiency.values, label="Ground Truth", color='tab:green', alpha=0.6, linewidth=2, linestyle='--')
    plt.plot(efficiency_pred_daily_m0.index, efficiency_pred_daily_m0, label="Reference", color='tab:red', linewidth=2, linestyle=':')
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    plt.title("Daily Efficiency: Model Estimation vs Ground Truth", fontsize=24)
    plt.xlabel("Day-Hour", fontsize=18)
    plt.xticks(fontsize=14)
    plt.ylabel("Power (W)", fontsize=18)
    plt.yticks(fontsize=14)
    plt.grid(True, axis='y')
    plt.legend(fontsize=12, frameon=True, facecolor='white', edgecolor='black')
    plt.show()


plot3days()
plotOutputEvolution()
plotEfficiencyEvolution()
